# Tuning

## Imports

In [1]:
import numpy as np
import pickle
import os

from scipy.stats import reciprocal
# Sklearn
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import RandomizedSearchCV

# Keras
import scikeras 
from scikeras.wrappers import KerasRegressor
import tensorflow as tf
import keras_tuner as kt




2023-06-04 00:44:19.765091: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-04 00:44:19.818935: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-04 00:44:19.819708: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-04 00:44:20.503068: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Get data

In [2]:

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42) # type: ignore
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

## Scikeras

In [3]:
param_distribs = {
    "model__n_hidden": list(range(0, 4)),
    "model__n_neurons": list(range(1, 101)),
    "model__learning_rate": reciprocal(3e-4, 3e-2).rvs(1000).tolist(),
    "model__optimizer": ['adam', 'sgd', 'nesterov']
}


In [4]:
def build_model(n_hidden, n_neurons, optimizer, learning_rate):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=X_train[0].shape))
    
    for layer in range(n_hidden):
        model.add(tf.keras.layers.Dense(n_neurons, activation="relu"))
    model.add(tf.keras.layers.Dense(1))
    
    if optimizer == "adam":
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer == "sgd":
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    elif optimizer == "nesterov":
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, nesterov=True)
        
    model.compile(loss="mse", optimizer=optimizer, metrics=["mse"])
    return model

## Early stopping

In [5]:
early_stopping = tf.keras.callbacks.EarlyStopping(patience=10, min_delta=1, verbose=1)


keras_reg = KerasRegressor(build_model, callbacks=[early_stopping])

## RandomizedSearchCV

In [6]:
rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3, verbose=2)
rnd_search_cv.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), verbose=0)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


2023-06-04 00:44:21.852574: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-04 00:44:21.852945: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 16: early stopping
121/121 [==============================] - 0s 1ms/step
[CV] END model__learning_rate=0.00048549849822155543, model__n_hidden=0, model__n_neurons=84, model__optimizer=nesterov; total time=   9.8s
Epoch 58: early stopping
121/121 [==============================] - 0s 1ms/step
[CV] END model__learning_rate=0.00048549849822155543, model__n_hidden=0, model__n_neurons=84, model__optimizer=nesterov; total time=  31.0s
Epoch 14: early stopping
121/121 [==============================] - 0s 1ms/step
[CV] END model__learning_rate=0.00048549849822155543, model__n_hidden=0, model__n_neurons=84, model__optimizer=nesterov; total time=   8.0s
Epoch 42: early stopping
121/121 [==============================] - 0s 1ms/step
[CV] END model__learning_rate=0.0017811972717947612, model__n_hidden=0, model__n_neurons=56, model__optimizer=adam; total time=  22.3s
Epoch 11: early stopping
121/121 [==============================] - 0s 1ms/step
[CV] END model__learning_rate=0.0017811972717

RandomizedSearchCV(cv=3,
                   estimator=KerasRegressor(callbacks=[<keras.callbacks.EarlyStopping object at 0x7f00ed852b90>], model=<function build_model at 0x7f00ee539080>),
                   param_distributions={'model__learning_rate': [0.004479938333026118,
                                                                 0.010026742036415225,
                                                                 0.0005780703207538449,
                                                                 0.0071418316450687835,
                                                                 0.0049023479745224224,
                                                                 0.004795070286442121,
                                                                 0.01649582231127...
                                                                 0.016593144198401487,
                                                                 0.0012512097823778575,
                                                                 0.0021517623599730555,
                                                                 0.0020452561049876753,
                                                                 0.0007739135658707072,
                                                                 0.021243132248952096,
                                                                 0.007697643098554799,
                                                                 0.017057533935944922, ...],
                                        'model__n_hidden': [0, 1, 2, 3],
                                        'model__n_neurons': [1, 2, 3, 4, 5, 6,
                                                             7, 8, 9, 10, 11,
                                                             12, 13, 14, 15, 16,
                                                             17, 18, 19, 20, 21,
                                                             22, 23, 24, 25, 26,
                                                             27, 28, 29, 30, ...],
                                        'model__optimizer': ['adam', 'sgd',
                                                             'nesterov']},
                   verbose=2)

Save to pkl

In [7]:
rnd_search_cv

RandomizedSearchCV(cv=3,
                   estimator=KerasRegressor(callbacks=[<keras.callbacks.EarlyStopping object at 0x7f00ed852b90>], model=<function build_model at 0x7f00ee539080>),
                   param_distributions={'model__learning_rate': [0.004479938333026118,
                                                                 0.010026742036415225,
                                                                 0.0005780703207538449,
                                                                 0.0071418316450687835,
                                                                 0.0049023479745224224,
                                                                 0.004795070286442121,
                                                                 0.01649582231127...
                                                                 0.016593144198401487,
                                                                 0.0012512097823778575,
                                                                 0.0021517623599730555,
                                                                 0.0020452561049876753,
                                                                 0.0007739135658707072,
                                                                 0.021243132248952096,
                                                                 0.007697643098554799,
                                                                 0.017057533935944922, ...],
                                        'model__n_hidden': [0, 1, 2, 3],
                                        'model__n_neurons': [1, 2, 3, 4, 5, 6,
                                                             7, 8, 9, 10, 11,
                                                             12, 13, 14, 15, 16,
                                                             17, 18, 19, 20, 21,
                                                             22, 23, 24, 25, 26,
                                                             27, 28, 29, 30, ...],
                                        'model__optimizer': ['adam', 'sgd',
                                                             'nesterov']},
                   verbose=2)

In [8]:
with open('rnd_search_params.pkl', 'wb') as f:
    pickle.dump(rnd_search_cv.best_params_, f)
    
with open('rnd_search_scikeras.pkl', 'wb') as f:
    pickle.dump(rnd_search_cv, f)

INFO:tensorflow:Assets written to: ram:///tmp/tmpl8o4qfa7/assets


INFO:tensorflow:Assets written to: ram:///tmp/tmpl8o4qfa7/assets


AttributeError: 'Adam' object has no attribute 'get_weights'

## Keras Tuner

In [9]:

def build_model(hp):
    n_hidden = hp.Int('n_hidden', min_value=0, max_value=3, default=2)
    n_neurons = hp.Int('n_neurons', min_value=1, max_value=100, default=30)
    learning_rate = hp.Choice('learning_rate', values=[1e-3, 3e-3, 1e-2, 3e-2])
    optimizer = hp.Choice('optimizer', values=['adam', 'sgd', 'nesterov'])

    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=X_train[0].shape))

    for layer in range(n_hidden):
        model.add(tf.keras.layers.Dense(n_neurons, activation="relu"))
    model.add(tf.keras.layers.Dense(1))

    if optimizer == "adam":
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer == "sgd":
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    elif optimizer == "nesterov":
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, nesterov=True)

    model.compile(loss="mse", optimizer=optimizer, metrics=["mse"])
    return model


## Create tuner

In [10]:
random_search_tuner = kt.RandomSearch(
build_model, objective="val_mse", max_trials=10, overwrite=True,
directory="my_california_housing", project_name="my_rnd_search", seed=42)

## Callback TensorBoard

In [11]:
root_logdir = os.path.join(random_search_tuner.project_dir, 'tensorboard')
tb = tf.keras.callbacks.TensorBoard(root_logdir)

## Run search

In [12]:
random_search_tuner.search(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), callbacks=[tb, early_stopping])

random_search_tuner.results_summary()


Trial 10 Complete [00h 00m 13s]
val_mse: 0.36270666122436523

Best val_mse So Far: 0.27389615774154663
Total elapsed time: 00h 02m 29s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Results summary
Results in my_california_housing/my_rnd_search
Showing 10 best trials
Objective(name="val_mse", direction="min")

Trial 01 summary
Hyperparameters:
n_hidden: 3
n_neurons: 36
learning_rate: 0.01
optimizer: adam
Score: 0.27389615774154663

Trial 07 summary
Hyperparameters:
n_hidden: 3
n_neurons: 33
learning_rate: 0.001
optimizer: adam
Score: 0.29524073004722595

Trial 03 summary
Hyperparameters:
n_hidden: 3
n_neurons: 10
learning_rate: 0.03
optimizer: adam
Score: 0.3073197901248932

Trial 02 summary
Hyperparameters:
n_hidden: 1
n_neurons: 25
learning_rate: 0.03
optimizer: sgd
Score: 0.33155351877212524

Trial 05 summary
Hyperparameters:
n_hidden: 3
n_neurons: 45
learning_rate: 0.003
optimizer: sgd
Score: 0.3415594696998596

Trial 08 summary
Hyperparameters:
n_hidden: 3
n_neurons: 46
learning_rate: 0.003
optimizer: nesterov
Score: 0.35678914189338684

Trial 09 summary
Hyperparameters:
n_hidden: 3
n_neurons: 15
learning_rate: 0.003
optimizer: sgd
Score: 0.36270666122436523


In [14]:
with open('kt_search_params.pkl', 'wb') as f:
    pickle.dump(random_search_tuner.get_best_hyperparameters()[0].values, f)

random_search_tuner.get_best_models(num_models=1)[0].save('kt_best_model.h5')
